# Fairness Indicators TensorBoard Plugin Example Colab

##Overview

In this activity, you'll use [Fairness Indicators for TensorBoard](https://github.com/tensorflow/tensorboard/tree/master/docs/fairness-indicators.md). With the plugin, you can visualize fairness evaluations for your runs and easily compare performance across groups.


# Importing

Run the following code to install the required libraries.

In [0]:
!pip install --upgrade pip
!pip install fairness-indicators
!pip install tensorflow_hub
!pip install tensorboard_plugin_fairness_indicators
# tensorboard_plugin_fairness_indicators has a dependency on tb-nightly and
# installing tb-nightly can cause a version conflict with tensorboard installed
# transitively from fairness-indicators. To avoid it, uninstall tensorboard
# before installing tb-nightly.
!pip uninstall -y tensorboard 
!pip install --upgrade tb-nightly

**Restart the runtime.** After the runtime is restarted, continue with following cells without running previous cell again.

In [0]:
%tensorflow_version 1.x

import datetime
import os
import tempfile
from tensorboard_plugin_fairness_indicators import summary_v2
import tensorflow as tf

# example_model.py is provided in fairness_indicators package to train and
# evaluate an example model. 
from fairness_indicators import example_model

tf.compat.v1.enable_eager_execution()

# Data and Constants

In [0]:
train_tf_file = tf.keras.utils.get_file('train.tf', 'https://storage.googleapis.com/civil_comments_dataset/train.tfrecord')
validate_tf_file = tf.keras.utils.get_file('validate.tf', 'https://storage.googleapis.com/civil_comments_dataset/validate.tfrecord')

BASE_DIR = tempfile.gettempdir()
TEXT_FEATURE = 'comment_text'
LABEL = 'toxicity'
FEATURE_MAP = {
    # Label:
    LABEL: tf.io.FixedLenFeature([], tf.float32),
    # Text:
    TEXT_FEATURE: tf.io.FixedLenFeature([], tf.string),

    # Identities:
    'sexual_orientation': tf.io.VarLenFeature(tf.string),
    'gender': tf.io.VarLenFeature(tf.string),
    'religion': tf.io.VarLenFeature(tf.string),
    'race': tf.io.VarLenFeature(tf.string),
    'disability': tf.io.VarLenFeature(tf.string),
}

# Train the Model

In [0]:
model_dir = os.path.join(BASE_DIR, 'train',
                         datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))

classifier = example_model.train_model(model_dir,
                                       train_tf_file,
                                       LABEL,
                                       TEXT_FEATURE,
                                       FEATURE_MAP)

# Run TensorFlow Model Analysis with Fairness Indicators
This step might take 2 to 5 minutes.

In [0]:
tfma_eval_result_path = os.path.join(BASE_DIR, 'tfma_eval_result')

example_model.evaluate_model(classifier,
                             validate_tf_file,
                             tfma_eval_result_path,
                             'gender',
                             LABEL,
                             FEATURE_MAP)

# Visualize Fairness Indicators in TensorBoard


Below you will visualize Fairness Indicators in Tensorboard and compare performance of each slice of the data on selected metrics. You can adjust the baseline comparison slice as well as the displayed threshold(s) using the drop down menus at the top of the visualization. You can also select different evaluation runs using the drop down menu at the top-left corner.

## Write Fairness Indicators Summary
Write summary file containing all required information to visualize Fairness Indicators in TensorBoard.

In [0]:
writer = tf.compat.v2.summary.create_file_writer(
    os.path.join(model_dir, 'fairness_indicators'))
with writer.as_default():
  summary_v2.FairnessIndicators(tfma_eval_result_path, step=1)
writer.close()

## Launch TensorBoard
Navigate to "Fairness Indicators" tab to visualize Fairness Indicators.

In [0]:
%load_ext tensorboard

In [0]:
%tensorboard --logdir=$model_dir